In [20]:
import tensorflow as tf
from pathlib import Path

In [ ]:
tf.config.list_physical_devices("GPU")

In [22]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
MODELS_DIR = ROOT_DIR / "models"

# 1. Data Collection

In [ ]:
from pathlib import Path
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg

In [24]:
DATA_DIR.mkdir(exist_ok=True)

# load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
# save to a file
with open(DATA_DIR / 'hamlet.txt', 'w') as file:
    file.write(data)

# 2. Data preprocessing

In [25]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from sklearn.model_selection import train_test_split

In [26]:
# load dataset
with open(DATA_DIR / 'hamlet.txt', 'r') as file:
    text = file.read().lower()

In [ ]:
# Tokenize the text - creating indexes for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

In [ ]:
tokenizer.word_index

In [ ]:
# Create input sequences
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

In [ ]:
# Implement pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences.shape

In [ ]:
# Create predictors and label
X, y = input_sequences[:, :-1], input_sequences[:, -1]
X, y

In [ ]:
# Convert indexes array to categorical array
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test

# 3. Build LSTM RNN Model

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.build(input_shape=(None, max_sequence_len - 1))

model.summary()

# 4. Train LSTM RNN Model

In [37]:
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=1)


# 5. Save LSTM RNN Model

In [50]:
import pickle

In [52]:
MODELS_DIR.mkdir(exist_ok=True)

model.save(MODELS_DIR / "predict_next_word_lstm.keras")

with open(MODELS_DIR / 'tokenizer.pickle', "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 6. Predict Next Word

In [41]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding="pre")
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
input_text = "Barn. Last night of all, When yond same"
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model=model, tokenizer=tokenizer, text=input_text, max_sequence_len=max_sequence_len)

print(f"Input text: {input_text}")
print(f"Next word: {next_word}")